## Eno-gastronomic Heritage Collection
Data integration and Data Profile Project - 02/2022   
Rachel Fanti Coelho Lima

## Summary
* 1. [Libraries](#libraries)
* 2. [Setup](#setup)
* 3. [Visualization](#visualization)   
     * 3.1 [Producers - location map](#producers_map)
     * 3.2 [Producers by grape variety - location map](#producers_by_grape_map)
     * 3.3 [Number of producers by Location](#producers_by_loc)
     * 3.4 [Percentage of wines by colour](#wines_by_colour)
     * 3.5 [Number of wines by certification](#wines_by_certification)
     * 3.6 [Number of wines by producer and locality](#wine_by_producer_locality)

## 1. Libraries <a class="anchor" id="libraries"></a>

In [49]:
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from SPARQLWrapper import SPARQLWrapper, JSON
import sparql_dataframe
import warnings
warnings.filterwarnings("ignore")

In [50]:
#pip install sparqlwrapper
#pip install plotly
#pip install "jupyterlab>=3" "ipywidgets>=7.6"
#pip install jupyter-dash

## 2. Setup <a class="anchor" id="setup"></a>

In [51]:
# Set up the endpoint and the URL (http://localhost:8080/sparql).

# SPARQLWrapper library https://rdflib.github.io/sparqlwrapper/ is used to send SPARQL queries and get results.    
# The following code gets the result as JSON documents and convert it to a Python dict object.

sparql = SPARQLWrapper("http://localhost:8080/sparql")

q1 = """

PREFIX : <http://www.semanticweb.org/rachel/ontologies/2022/0/untitled-ontology-30#>

SELECT ?cod ?wn ?sour ?col ?alc ?org_desc ?p ?lat ?long ?lau ?reg ?prov
WHERE {?w :isProducedBy ?ac;
:wWineCode ?cod;
:wName ?wn;
:hasDescription ?d.
?d :wdColour ?col.
?d:hasOrganolepticDescription ?od.
?ac a :Actor;
:hasMainAddress ?ad;
:acName ?p.
?ad :hasGeolocalization ?g.
?g :gLatitude ?lat;
:gLongitude ?long.
?ad :hasMunicipality ?m.
?m :mLauNameNational ?lau;
:mNUTSLevel2 ?reg;
:mNUTSLevel3 ?prov.
OPTIONAL {?w:wSource ?sour.}
OPTIONAL {?d :wdAlcoholContent ?alc.}
OPTIONAL {?od :wodOrganolepticDescription ?org_desc.}
}

"""

#sparql.setQuery(q1)
#sparql.setReturnFormat(JSON)
#results = sparql.query().convert()
#print(results)

# The SPARQL results are converted to a pandas DataFrame for data analysis.    
# The library sparql-dataframe https://github.com/lawlesst/sparql-dataframe is handy for this.

endpoint = "http://localhost:8080/sparql"

#dataset of wine
df_w = sparql_dataframe.get(endpoint, q1)
df_w.head()

,cod,wn,sour,col,alc,org_desc,p,lat,long,lau,reg,prov
0,A' Scippata,Costa d'Amalfi,089 856209,Rosso,0.14,"Colore rosso intenso, dai rilessi violacei. Pr...",Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
1,Colle Santa Marina,Costa d'Amalfi,089 856209,Bianco,"13,5%",Vino dal colore giallo paglierino. Delicato e ...,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
2,Passion,Colli di Salerno,089 856209,Bianco,"14,5%",Vino passito dal colore paglierino carico con ...,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
3,Piedirosso,Colli di Salerno,089 856209,Rosso,"11,5%",Vino dal colore rosso rubino con netti rifless...,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
4,Rosato,Costa d'Amalfi,089 856209,Rosato,"12,5%",Si presenta di colore rosato con tonalitÃ lil...,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno


In [52]:
q2 = """

PREFIX : <http://www.semanticweb.org/rachel/ontologies/2022/0/untitled-ontology-30#>

SELECT ?cod ?wn ?sour ?col ?alc ?org_desc ?perc ?grap ?p ?lat ?long ?lau ?reg ?prov
WHERE {?w :isProducedBy ?ac;
:wWineCode ?cod;
:wName ?wn;
:hasDescription ?d.
?d :wdColour ?col.
?d:hasOrganolepticDescription ?od.
?ac a :Actor;
:hasMainAddress ?ad;
:acName ?p.
?ad :hasGeolocalization ?g.
?g :gLatitude ?lat;
:gLongitude ?long.
?ad :hasMunicipality ?m.
?m :mLauNameNational ?lau;
:mNUTSLevel2 ?reg;
:mNUTSLevel3 ?prov.
OPTIONAL {?w:wSource ?sour.}
OPTIONAL {?d :wdAlcoholContent ?alc.}
OPTIONAL {?od :wodOrganolepticDescription ?org_desc.}
?w :hasGrapeComposition ?gc.
?gc :hasGrape ?gv.
?gv :gvName ?grap. optional {?gc :wgcPercentageOfGrape ?perc.}
}

"""
#dataset of grape_composition 
#(only data for the wines that has grape_composition, more than one line per wine, since they can have more than one grape) )
df_g = sparql_dataframe.get(endpoint, q2)
df_g.head()

,cod,wn,sour,col,alc,org_desc,perc,grap,p,lat,long,lau,reg,prov
0,Piedirosso,Colli di Salerno,089 856209,Rosso,"11,5%",Vino dal colore rosso rubino con netti rifless...,15%,Altri,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
1,Passion,Colli di Salerno,089 856209,Bianco,"14,5%",Vino passito dal colore paglierino carico con ...,20%,Biancatenera,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
2,Passion,Colli di Salerno,089 856209,Bianco,"14,5%",Vino passito dal colore paglierino carico con ...,40%,Biancazita,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
3,Colle Santa Marina,Costa d'Amalfi,089 856209,Bianco,"13,5%",Vino dal colore giallo paglierino. Delicato e ...,20%,Biancolella,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno
4,Colle Santa Marina,Costa d'Amalfi,089 856209,Bianco,"13,5%",Vino dal colore giallo paglierino. Delicato e ...,40%,Falanghina,Giuseppe Apicella,40.706354,14.622361,Tramonti,Campania,Salerno


In [53]:
q3 = """

PREFIX : <http://www.semanticweb.org/rachel/ontologies/2022/0/untitled-ontology-30#>

SELECT ?w ?c ?cert ?ext_cert
WHERE {?w a :Wine;
:hasCertification ?c. 
?c :cName ?cert;
:cExtendedCode ?ext_cert.
}

"""
# dataset of certificates
#(only data for the wines that has certfication, it can have more than one line per wine) 
df_c = sparql_dataframe.get(endpoint, q3)
df_c.head()

,w,c,cert,ext_cert
0,http://www.semanticweb.org/rachel/ontologies/2...,http://www.semanticweb.org/rachel/ontologies/2...,D.O.C.G.,Denominazione di Origine Controllata e Garantita
1,http://www.semanticweb.org/rachel/ontologies/2...,http://www.semanticweb.org/rachel/ontologies/2...,I.G.T.,Indicazione Geografica Tipica (IGT)
2,http://www.semanticweb.org/rachel/ontologies/2...,http://www.semanticweb.org/rachel/ontologies/2...,D.O.C.G.,Denominazione di Origine Controllata e Garantita
3,http://www.semanticweb.org/rachel/ontologies/2...,http://www.semanticweb.org/rachel/ontologies/2...,D.O.C.,Denominazione di Origine Controllata
4,http://www.semanticweb.org/rachel/ontologies/2...,http://www.semanticweb.org/rachel/ontologies/2...,D.O.C.G.,Denominazione di Origine Controllata e Garantita


In [54]:
q4 = """

PREFIX : <http://www.semanticweb.org/rachel/ontologies/2022/0/untitled-ontology-30#>

SELECT ?p ?r ?lat ?long ?lau ?reg ?prov
WHERE {?ac :hasMainAddress ?d;
:acName ?p.
?d :hasGeolocalization ?g.
?g :gLatitude ?lat;
:gLongitude ?long.
?d :hasMunicipality ?m.
?m :mLauNameNational ?lau;
:mNUTSLevel2 ?reg;
:mNUTSLevel3 ?prov.

}

"""
# dataset for producer
df_p = sparql_dataframe.get(endpoint, q4)
df_p.head()

,p,r,lat,long,lau,reg,prov
0,A Casa,NaN,40.930863,14.824782,Avellino,Campania,Avellino
1,Agnanum,NaN,40.839236,14.161481,Napoli,Campania,Napoli
2,Agricola Boccella,NaN,40.927652,15.052481,Castelfranci,Campania,Avellino
3,Agricola San Domenico,NaN,40.731223,13.860394,Forio,Campania,Napoli
4,Agricola San Teodoro,NaN,41.337258,13.919779,Galluccio,Campania,Caserta


## 3. Visualization <a class="anchor" id="visualization"></a>

### 3.1 Producers - location map  <a class="anchor" id="producers_map"></a>

In [55]:
fig = px.scatter_mapbox(df_w, lat="lat", lon="long", color = "prov", hover_name="p", hover_data=["lau", "prov"], zoom=6.5) 
fig.update_layout(mapbox_style="open-street-map") #carto-positron, open-street-map
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.update_geos(fitbounds = 'locations')
fig.show()

### 3.2 Producers by grape variety - location map <a class="anchor" id="producers_by_grape_map"></a>

In [56]:
grapes = sorted(df_g['grap'].unique())

app = JupyterDash(__name__)

server = app.server

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown_grape",
        options=[{"label": x, "value": x} for x in grapes],
        value=grapes[0],
        clearable=False,
    ),
    dcc.Graph(id="map_producers"),
    dcc.Textarea(
        #placeholder="Enter a value",
        value="The map shows only producers who have provided information about grape variety.",
        style={'width': '86.5%'}
    )
])

@app.callback(
    Output("map_producers", "figure"), 
    [Input("dropdown_grape", "value")])

def display_map_producers(dropdown_grape):
    df1 = df_g[df_g['grap'] == dropdown_grape]
    fig = px.scatter_mapbox(df1, lat="lat", lon="long", color = "prov", hover_name="p", hover_data=["lau", "prov"], zoom=6.5)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
    fig.update_geos(fitbounds = 'locations')
    return fig
    
#app.run_server(port=8051)
app.run_server('inline')

### 3.3 Number of producers by Location <a class="anchor" id="producers_by_loc"></a>

In [57]:
list_local = ['lau', 'prov', 'reg']

app = JupyterDash(__name__)

server = app.server

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown_locality",
        options=[{"label": x, "value": x} for x in list_local],
        value=list_local[0],
        clearable=False,
    ),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown_locality", "value")])
def update_bar_chart(local):
    fig = px.histogram(df_p, x=local).update_xaxes(categoryorder="total descending")
    return fig


app.run_server('inline')

### 3.4 Percentage of wines by colour <a class="anchor" id="wines_by_colour"></a>

In [58]:
df_col = df_w['col'].value_counts()
df_col = df_col.to_frame()

In [59]:
fig = px.pie(df_col, values='col', names=df_col.index, title='Percentage of wines by colour')
fig.update_traces(textposition='inside', textinfo='percent+value+label')
fig.show()


### 3.5 Number of wines by certification <a class="anchor" id="wines_by_certificate"></a>

In [60]:
fig = px.histogram(df_c, x="cert").update_xaxes(categoryorder="total descending", title='Number of wines by certification')
fig.show()

### 3.6 Number of wines by producer and locality <a class="anchor" id="wines_by_producer_locality"></a>

In [61]:
list_local = ['lau', 'prov', 'reg']
list_values = sorted(df_g['prov'].unique())

app = JupyterDash(__name__)

server = app.server

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown_locality",
        options=[{"label": x, "value": x} for x in list_local],
        value=list_local[1],
        clearable=False,
    ),
    dcc.Dropdown(
        id="dropdown_chosen_locality",
        value=list_values[0],
        clearable=False,
    ),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("dropdown_chosen_locality", 'options'), 
    Input("dropdown_locality", "value"))

def update_dropdown (locality):
    if locality=='lau':
        list_values = sorted(df_g['lau'].unique())
    elif locality=='prov':
        list_values = sorted(df_g['prov'].unique())
    else:
        list_values = sorted(df_g['reg'].unique())
    return  [{'label': i, 'value': i} for i in list_values]

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown_locality", "value"),
    Input("dropdown_chosen_locality", "value")])

def update_bar_chart(locality, chosen_locality):
    df2 = df_w[df_w[locality]==chosen_locality]
    fig = px.histogram(df2, x='p').update_xaxes(categoryorder="total descending")
    return fig

app.run_server('inline')
#app.run_server(port=8050)